In [6]:
# set the environment path to find Recommenders
import sys
sys.path.append("..")

In [ ]:
from utilities.recommender.sar import sar_singlenode

SyntaxError: only named arguments may follow *expression (sar_pyspark.py, line 388)